## Необходимые пакеты:

In [39]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import chromedriver_binary 
import urllib
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import lxml
import requests
import base64
import json
import pandas as pd
import numpy as np
from secrets import *
import urllib.request
from fake_useragent import UserAgent
from bs4 import BeautifulSoup 
from selenium.webdriver.common.action_chains  import ActionChains
import random

#####  Подготовка к работе с API, авторизируемся на сайте и изменяем кодировку для получения токена доступа.

In [40]:
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}


message = f"{'3e661d7345d04909b518ec04ec996fd2'}:{'c3ce130028654a779d6404a619845bd6'}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']

In [41]:
token

'BQAD2CtWhOULIaq_fc_o-aSG-8q5E857QAr877lpDdBIFletR_rRkKI3eB-cQXlOuev6NSHbt2UPpbmb01w'

Успешно получен токен.

###### Функция получения сведений о треках, находящихся в одном плейлисте

In [42]:
def get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = playlist_req_to_list (page)    
    return page


###### Функция по вытягиванию параметров песни

In [43]:
def song_paramets(song_id):
    
    song_url = f'https://api.spotify.com/v1/audio-features/{song_id}'
    headers = {
        "Authorization": "Bearer " + token
    }

    page = requests.get(url=song_url, headers=headers)
    page =  page.json()
    return page

###### Функция распредения инфомации из json-формата в DataFrame-таблицу


In [44]:
def playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
    duration = [None for i in range(50)]
    artist_name = [None for i in range(50)]
    artist_id = [None for i in range(50)]
    album_name = [None for i in range(50)]
    album_id = [None for i in range(50)]
    release_date = [None for i in range(50)]
    local_position = [None for i in range(50)]
    world_position = [None for i in range(50)]
    danceability = [None for i in range(50)]
    energy = [None for i in range(50)]
    key = [None for i in range(50)]
    loudness = [None for i in range(50)]
    mode = [None for i in range(50)]
    speechiness = [None for i in range(50)]
    acousticness = [None for i in range(50)]
    instrumentalness = [None for i in range(50)]
    liveness = [None for i in range(50)]
    valence = [None for i in range(50)]
    tempo = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        duration[i] = item['track']['duration_ms']
        local_position[i] = item['track']['popularity']       
        artist_name[i] = item['track']['album']['artists'][0]['name']
        artist_id[i] = item['track']['album']['artists'][0]['id']
        album_name[i] = item['track']['album']['name']
        album_id[i] = item['track']['album']['id']
        release_date[i] = item['track']['album']['release_date']
        
        parametrs = song_paramets(song_id[i])
        
        danceability[i] = parametrs['danceability']
        energy[i] = parametrs['energy']
        key[i] = parametrs['key']
        loudness[i] = parametrs['loudness']
        mode[i] = parametrs['mode']
        speechiness[i] = parametrs['speechiness']
        acousticness[i] = parametrs['acousticness']
        instrumentalness[i] = parametrs['instrumentalness']
        liveness[i] = parametrs['liveness']
        valence[i] = parametrs['valence']
        tempo[i] = parametrs['tempo']
        
        i += 1
        
    data_dict = {'song_name' : song_name,
                 'song_id' : song_id,
                 'duration_ms' : duration,
                 'local_position' : local_position,
                 'artist_name' : artist_name,
                 'artist_id' : artist_id,
                 'album_name' : album_name,
                 'album_id' : album_id,
                 'release_date' : release_date,
                 'danceability': danceability,
                 'energy': energy,
                 'key': key,
                 'loudness': loudness,
                 'mode': mode,
                 'speechiness': speechiness,
                 'acousticness': acousticness,
                 'instrumentalness': instrumentalness,
                 'liveness': liveness,
                 'valence': valence,
                 'tempo': tempo
                }
    data_set = pd.DataFrame(data_dict)
    
    return data_set

##### Функция по получению основной инфы о плейлисте

In [45]:
def playlist_info(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=song_url, headers=headers)
    page =  page.json()
    text = page['description']
    number = page['followers']['total']
    country = text[text.find('now -')+6:-1]
    info = [country,number]
    return info

## Находим все айдишники альбомов 
#### Полученное ids-список айдишников

In [59]:
site = "https://open.spotify.com/genre/charts-regional-weekly"

#chrome_options = Options() работает у Дениса
#chrome_options.add_argument("--headless")   настройка на всякий случай

#настраиваем селениум
#driver = webdriver.Chrome('D:\chromedriver.exe') работает у Дениса
driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver') #работает у Сани
#открываем сайт, двигаем мышку и кликаем, чтобы потом заработала прокрутка
driver.get(site)
driver.maximize_window()
time.sleep(3)
ActionChains(driver).move_by_offset(2000, 900).click().perform() #у дениса 1500 350
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) 
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) #моя добавка
time.sleep(3)


#получаем код страницы и делаем суп
response = driver.page_source
driver.close()
tree = BeautifulSoup(response, 'html.parser')

In [61]:
#ищем все ссылки на странице
a = tree.find_all('a', class_ = 'f7ebc3d96230ee12a84a9b0b4b81bb8f-scss')

#из блоков с ссылками достаем айдишники
ids = []
for i in a:
    id = i.get('href')
    ids.append(id[10:])

##### Функция по поиску прослушиваний для каждого плейлиста


In [65]:
def viewslist(playlist_id):
    site = f'https://open.spotify.com/playlist/{playlist_id}'
    driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver')
    driver.get(site)
    time.sleep(2)
    response = driver.page_source
    driver.close()
    tree = BeautifulSoup(response, 'html.parser')
    a = tree.find_all('div', class_ = 'd47b790d001ed769adcd9ddfc0e83acc-scss f3fc214b257ae2f1d43d4c594a94497f-scss')
    res = []
    b = list(a)
    for i in b:
        views = str(i)[-16:-6].replace('\xa0', '')
        res.append(views)
    views = []
    for i in res:
        ddd = [s for s in i.split('>') if s.isdigit()]
        views.append(int(ddd[0]))
    return views

# Cобираем датасэты

## Структура данных:
#### Маленький датасетик вида (айди листа, страна, кол-во лайков)
#### Датасет побольбше (данные о каждой песне)
#### Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

### Маленький датасетик